# Notre Projet : I-Un scoring pour choisir nos investissements 

In [ ]:
# Le but du projet est d'étudier les ratios financiers classiques que l'on trouve sur Yahoo Finance
#afin d'évaluer les entreprises et de sélectionner celles qui remplissent le plus de "bons" critères
# Nous définissons "bons" pour chaque critère tout au long du code grâce à cet article : https://www.labsterx.com/blog/fundamental-analysis-using-yahoo-finance/
# Nous nous sommes concentrés sur les entreprises tech
# Le but final est donc d'attribuer un score pour chaque entreprise et de regarder celles qui ont le plus haut score
#ou le score le plus équilibré pour faire nos investissements.

Importation des bibliothèques nécessaires au code 

In [181]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Création d'un dataframe adapté aux manipulations 

In [182]:
#Les données sont scrappées en format EXCEL puis téléchargées depuis l'ordinateur de l'utilisateur
data0= pd.read_excel(r"H:\Documents\ENSAE\1er semestre 2a\Python DS\projet\donnees_python.xlsx")
print(data0)

      Symbol Market Cap (intraday)  Enterprise Value  Trailing P/E   \
0       AAPL                  2.94T             3.01T         31.99   
1       MSFT                  2.57T             2.50T         38.32   
2       NVDA                754.95B           747.48B         93.06   
3        TSM                567.15B           553.17B          30.2   
4       ASML                319.75B           319.36B         52.21   
..       ...                    ...               ...           ...   
500    SABRP                    NaN               NaN           NaN   
501    IIVIP                    NaN               NaN           NaN   
502      IOT                    NaN               NaN           NaN   
503  CLVT-PA                    NaN               NaN           NaN   
504    AVGOP                    NaN               NaN           NaN   

    Forward P/E   PEG Ratio (5 yr expected)   Price/Sales (ttm)  \
0          32.05                        3.85               8.27   
1          37

In [183]:
data = data0.copy()

In [184]:
data.columns

Index(['Symbol', 'Market Cap (intraday) ', 'Enterprise Value ',
       'Trailing P/E ', 'Forward P/E ', 'PEG Ratio (5 yr expected) ',
       'Price/Sales (ttm)', 'Price/Book (mrq)', 'Enterprise Value/Revenue ',
       'Enterprise Value/EBITDA ',
       ...
       'Shares Short (Jun 29, 2021) 4', 'Short Ratio (Jun 29, 2021) 4',
       'Short % of Float (Jun 29, 2021) 4',
       'Short % of Shares Outstanding (Jun 29, 2021) 4',
       'Shares Short (prior month May 27, 2021) 4',
       'Shares Short (Apr 29, 2021) 4', 'Short Ratio (Apr 29, 2021) 4',
       'Short % of Float (Apr 29, 2021) 4',
       'Short % of Shares Outstanding (Apr 29, 2021) 4',
       'Shares Short (prior month Mar 30, 2021) 4'],
      dtype='object', length=136)

In [186]:
for c in data.columns :
    if "Short" in c :
        del data[c]

In [187]:
for c in data.columns :
    if "Ratio" in c :
        print(c)
        

PEG Ratio (5 yr expected) 
Payout Ratio 4
Current Ratio (mrq)


In [188]:
data

,Symbol,Market Cap (intraday),Enterprise Value,Trailing P/E,Forward P/E,PEG Ratio (5 yr expected),Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue,Enterprise Value/EBITDA,...,Diluted EPS (ttm),Quarterly Earnings Growth (yoy),Total Cash (mrq),Total Cash Per Share (mrq),Total Debt (mrq),Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm),Levered Free Cash Flow (ttm)
0,AAPL,2.94T,3.01T,31.99,32.05,3.85,8.27,46.67,8.22,24.41,...,5.61,0.622,62.64B,3.82,136.52B,216.39,1.08,3.84,104.04B,73.3B
1,MSFT,2.57T,2.50T,38.32,37.45,2.91,14.75,16.92,14.21,27.81,...,8.94,0.476,130.58B,17.39,78.93B,51.94,2.16,20.24,81.95B,49.82B
2,NVDA,754.95B,747.48B,93.06,58.82,3.22,31.51,31.72,30.79,75.3,...,3.24,0.844,19.3B,7.72,11.83B,49.70,7.14,9.51,8.14B,5.94B
3,TSM,567.15B,553.17B,30.2,25.38,1.89,11.31,8.23,0.37,0.53,...,3.97,0.138,989.53B,190.80,605.18B,29.12,2.09,400.30,993.09B,103.62B
4,ASML,319.75B,319.36B,52.21,40.82,1.81,15.94,24.77,17.86,47.7,...,14.84,0.639,4.46B,10.92,4.11B,35.69,1.71,28.20,9.1B,6.56B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,SABRP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.5B,4.73,4.82B,1328.39,3.04,NaN,-773.18M,-331.37M
501,IIVIP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.609,1.56B,14.70,1.55B,37.04,2.97,NaN,492.36M,275.67M
502,IOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
503,CLVT-PA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
#Observation des colonnes qu'il faudra modifier car leurs valeurs sont exprimés avec des B (pour billions) ou des M (pour millions)
for c in data.columns :
    try :
        data[c] = data[c].map(lambda x : float(str(x).replace("k","")))
    except :
        print("colonne pas utilisable ", c)

colonne pas utilisable  Symbol
colonne pas utilisable  Market Cap (intraday) 
colonne pas utilisable  Enterprise Value 
colonne pas utilisable  Avg Vol (3 month) 3
colonne pas utilisable  Avg Vol (10 day) 3
colonne pas utilisable  Shares Outstanding 5
colonne pas utilisable  Implied Shares Outstanding 6
colonne pas utilisable  Float 8
colonne pas utilisable  Dividend Date 3
colonne pas utilisable  Ex-Dividend Date 4
colonne pas utilisable  Last Split Factor 2
colonne pas utilisable  Last Split Date 3
colonne pas utilisable  Fiscal Year Ends 
colonne pas utilisable  Most Recent Quarter (mrq)
colonne pas utilisable  Revenue (ttm)
colonne pas utilisable  Gross Profit (ttm)
colonne pas utilisable  EBITDA 
colonne pas utilisable  Net Income Avi to Common (ttm)
colonne pas utilisable  Total Cash (mrq)
colonne pas utilisable  Total Debt (mrq)
colonne pas utilisable  Operating Cash Flow (ttm)
colonne pas utilisable  Levered Free Cash Flow (ttm)


In [190]:
c = "Total Cash (mrq)"

In [191]:
#Il s'agit ici de tenter de remplacer les valeurs avec B ou M ou T par les chiffres dans la colonne Total Cash
#on réutilise cette commande plus tard pour d'autres colonnes
data[c] = np.where(data[c].str.contains("M"),data[c].map(lambda x : 1000000*float(str(x).replace("M","")) if ("M" in str(x)) else x),data[c])
data[c] = np.where(data[c].str.contains("B"),data[c].map(lambda x : 1000000000*float(str(x).replace("B","")) if ("B" in str(x)) else x),data[c])
data[c] = np.where(data[c].str.contains("T"),data[c].map(lambda x : 1000000000000*float(str(x).replace("T","")) if ("T" in str(x)) else x),data[c])

In [192]:
#Prévisualisation du data frame obtenu
data.head(32)

,Symbol,Market Cap (intraday),Enterprise Value,Trailing P/E,Forward P/E,PEG Ratio (5 yr expected),Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue,Enterprise Value/EBITDA,...,Diluted EPS (ttm),Quarterly Earnings Growth (yoy),Total Cash (mrq),Total Cash Per Share (mrq),Total Debt (mrq),Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm),Levered Free Cash Flow (ttm)
0,AAPL,2.94T,3.01T,31.99,32.05,3.85,8.27,46.67,8.22,24.41,...,5.61,0.622,6.264e+10,3.82,136.52B,216.39,1.08,3.84,104.04B,73.3B
1,MSFT,2.57T,2.50T,38.32,37.45,2.91,14.75,16.92,14.21,27.81,...,8.94,0.476,1.3058e+11,17.39,78.93B,51.94,2.16,20.24,81.95B,49.82B
2,NVDA,754.95B,747.48B,93.06,58.82,3.22,31.51,31.72,30.79,75.30,...,3.24,0.844,1.93e+10,7.72,11.83B,49.70,7.14,9.51,8.14B,5.94B
3,TSM,567.15B,553.17B,30.20,25.38,1.89,11.31,8.23,0.37,0.53,...,3.97,0.138,9.8953e+11,190.80,605.18B,29.12,2.09,400.30,993.09B,103.62B
4,ASML,319.75B,319.36B,52.21,40.82,1.81,15.94,24.77,17.86,47.70,...,14.84,0.639,4.46e+09,10.92,4.11B,35.69,1.71,28.20,9.1B,6.56B
5,AVGO,260.01B,289.39B,46.79,20.66,1.44,10.19,10.68,10.54,19.70,...,15.00,0.502,1.216e+10,29.46,40.27B,161.16,2.64,60.44,13.76B,11.87B
6,ADBE,269.35B,267.87B,46.86,39.68,2.05,18.04,18.69,16.97,40.55,...,10.02,-0.452,5.8e+09,12.18,4.67B,31.58,1.25,31.09,NaN,NaN
7,ACN,239.67B,235.00B,41.42,37.31,3.52,4.85,12.27,4.65,24.20,...,9.61,0.194,5.64e+09,8.93,3.48B,16.61,1.22,32.20,7.9B,6.32B
8,CSCO,249.89B,236.05B,22.03,17.24,3.15,4.94,5.85,4.65,14.23,...,2.69,0.371,2.335e+10,5.54,10.69B,25.03,1.62,10.13,14.79B,10.93B
9,ORCL,280.56B,323.97B,21.88,21.79,2.78,7.44,NaN,7.83,22.68,...,3.54,NaN,2.284e+10,8.55,78.43B,NaN,1.65,-3.78,10.25B,9.68B


In [193]:
# On remplace 2.80k par 2.80 dans la colonne Trailing PE ligne 31 (erreur de Yahoo Finance dans sa database)
data['Trailing P/E '] = data['Trailing P/E '].replace("2.80k","2.80")

Création de sous data frames afin de créer des sous catégories en fonctions de ratios prédéfinis. 

Premier sous data frames avec les ratios: Trailing P/E & Forward P/E

In [194]:
#D'après les traders, un P/E négatif est mauvais signe donc LYFT ne peut pas être considéré comme prometteur avec cette analyse
PEdf=data[['Trailing P/E ', 'Symbol']]
PEdf = PEdf[~PEdf['Trailing P/E '].isna()]
print(PEdf.sort_values(by=[ 'Trailing P/E ', 'Symbol'], ascending=True))


     Trailing P/E  Symbol
129           0.90   LYFT
115           1.50   CDAY
42            1.97   DDOG
31            2.80   WDAY
85            3.19     ZI
..             ...    ...
111         220.54     DT
33          290.28    FIS
60          473.50   PLTR
104         488.00   NUAN
20          593.58    NOW

[142 rows x 2 columns]


In [195]:
# Conformément à la presse spécialisée, les entreprises à privilégier suivant ce ratio sont celles avec un Trailing P/E inf ou égal à 59
# 59 correspond ici à la valeur moyenne tirée du jeu de données en notre possession
PEdf['Trailing P/E '].mean()


59.03535211267606

In [196]:
# C'est bon signe que Forward P/E est inférieur au trailing P/E, car cela veut dire que l'entreprise est plus enclain à augmenter ses revenus l'an prochain.
PEFdf=data[['Forward P/E ', 'Symbol']]
PEFdf = PEFdf[~PEFdf['Forward P/E '].isna()]
print(PEFdf)

     Forward P/E  Symbol
0           32.05   AAPL
1           37.45   MSFT
2           58.82   NVDA
3           25.38    TSM
4           40.82   ASML
..            ...    ...
184         17.48   LITE
185         28.49   NVEI
186         24.21   LFUS
187         16.81   IIVI
188         30.96   OLED

[159 rows x 2 columns]


In [197]:
#Les deux data frames obtenues sont donc fusionnées (ie PEdf et PEFdf) pour vérifier l'hypothèse précédente (ie forward P/E < trailing P/E) et rester avec l'ordre de PEdf
dfPEfusion=pd.merge(PEdf,PEFdf,how="left",on="Symbol")
dfPEfusion

,Trailing P/E,Symbol,Forward P/E
0,31.99,AAPL,32.05
1,38.32,MSFT,37.45
2,93.06,NVDA,58.82
3,30.20,TSM,25.38
4,52.21,ASML,40.82
...,...,...,...
137,19.02,LITE,17.48
138,111.77,NVEI,28.49
139,26.00,LFUS,24.21
140,26.82,IIVI,16.81


In [198]:
dfPEfusion =dfPEfusion.sort_values(by=['Trailing P/E '])

In [199]:
dfPEfusion = dfPEfusion[~dfPEfusion['Forward P/E '].isna()]
print(dfPEfusion)

     Trailing P/E  Symbol  Forward P/E 
93            1.50   CDAY        227.27
38            1.97   DDOG        357.14
28            2.80   WDAY         77.52
69            3.19     ZI         90.91
81            5.90    HPE          7.42
..             ...    ...           ...
135         203.42   SLAB         87.72
89          220.54     DT         75.19
30          290.28    FIS         13.93
52          473.50   PLTR         92.59
20          593.58    NOW         88.50

[137 rows x 3 columns]


In [200]:
#Création d'un booléen pour trouver l'ensemble des entreprises qui vérifient la condition énoncée pour ce ratio. 
np.where(dfPEfusion['Trailing P/E ']>dfPEfusion['Forward P/E '], True, False)

array([False, False, False, False, False,  True, False, False, False,
        True, False,  True,  True,  True, False,  True, False,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True, False,  True,  True,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True, False,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [201]:
#on crée une colonne avec l'ensemble des booléens qui vérifient la condition Trailing P/E > Forward P/E vraie ou fausse
dfPEfusion['conditionT/F']=np.where(dfPEfusion['Trailing P/E ']>dfPEfusion['Forward P/E '], True, False)

In [202]:
dfPEfusion

,Trailing P/E,Symbol,Forward P/E,conditionT/F
93,1.50,CDAY,227.27,False
38,1.97,DDOG,357.14,False
28,2.80,WDAY,77.52,False
69,3.19,ZI,90.91,False
81,5.90,HPE,7.42,False
...,...,...,...,...
135,203.42,SLAB,87.72,True
89,220.54,DT,75.19,True
30,290.28,FIS,13.93,True
52,473.50,PLTR,92.59,True


In [203]:
#on vérifie le premier critère : Trailing P/E <= 59 à l'aide de booléens
dfPEfusion['conditionmoyPET']=np.where(dfPEfusion['Trailing P/E ']<=59, True, False)

In [204]:
##on obtient notre df avec toutes les entreprises qui vérifient les deux conditions avantageuses, les premières étant les meilleures suivant ces deux ratios
dfPEfusion['conditionTrailingPE']=np.where((dfPEfusion['conditionT/F'] == True) & (dfPEfusion['conditionmoyPET'] ==True), True, False)
dfPEfusion
del dfPEfusion['conditionT/F']
del dfPEfusion ['conditionmoyPET']
dfPEfusion


,Trailing P/E,Symbol,Forward P/E,conditionTrailingPE
93,1.50,CDAY,227.27,False
38,1.97,DDOG,357.14,False
28,2.80,WDAY,77.52,False
69,3.19,ZI,90.91,False
81,5.90,HPE,7.42,False
...,...,...,...,...
135,203.42,SLAB,87.72,False
89,220.54,DT,75.19,False
30,290.28,FIS,13.93,False
52,473.50,PLTR,92.59,False


Deuxième sous data frame avec le ratio: 'PEG Ratio (5yr expected)'

In [205]:
# Pour certains investisseurs un PEG Ratio (5 yr expected) plus bas que 1 est attractif (ie le cour de l'action est sous-évalué) et un PEG Ratio (5 yr expected)  supérieur à 2 indique qu'une action est surévalué.
#Pour comparer les entreprises il convient donc de les classer en fonction de leur PEG Ratio afin de pouvoir ordonner notre data frame

dfPEG=data[['PEG Ratio (5 yr expected) ', 'Symbol']]
dfPEG = dfPEG[~dfPEG['PEG Ratio (5 yr expected) '].isna()]
dfPEG=dfPEG.sort_values(by=[ 'PEG Ratio (5 yr expected) ', 'Symbol'], ascending=True)
print(dfPEG)

     PEG Ratio (5 yr expected)  Symbol
172                        0.33    ARW
107                        0.40    WDC
78                         0.56     ON
116                        0.67   NLOK
34                         0.73   COIN
..                          ...    ...
27                         8.78   TEAM
106                       12.06   SPLK
53                        12.14     ZS
90                        45.50   PATH
64                       261.45   OKTA

[116 rows x 2 columns]


In [206]:
#Création d'une colonne sur laquelle on vérifie les conditions à l'aide de booléens
dfPEG['valeurPEG<1']= np.where(dfPEG['PEG Ratio (5 yr expected) ']<1, True, False)

In [207]:
dfPEG

,PEG Ratio (5 yr expected),Symbol,valeurPEG<1
172,0.33,ARW,True
107,0.40,WDC,True
78,0.56,ON,True
116,0.67,NLOK,True
34,0.73,COIN,True
...,...,...,...
27,8.78,TEAM,False
106,12.06,SPLK,False
53,12.14,ZS,False
90,45.50,PATH,False


Troisième sous data frame pour le ratio : Price/Sales (ttm)

In [208]:
#Pour beaucoup d'investisseurs le ratio P/S (Price/Sales) est très importants car il est neutre et ne dépend pas de l'action humaine afin de le rendre plus attractif.
#Notre critère de comparaison ici sera les entreprise avec un ratio Prices/Sales (ttm) très bas : plus il est bas plus il faut investir dans l'entreprise
dfPS=data[['Price/Sales (ttm)', 'Symbol']]
dfPS = dfPS[~dfPS['Price/Sales (ttm)'].isna()]
dfPS=dfPS.sort_values(by=[ 'Price/Sales (ttm)', 'Symbol'], ascending=True)
print(dfPS)

     Price/Sales (ttm) Symbol
150               0.21    SNX
172               0.28    ARW
178               0.34   FLEX
162               0.34    JBL
57                0.43   DELL
..                 ...    ...
42               60.91   DDOG
83               71.71   BILL
55               73.49    NET
127              96.85      S
22              106.51   SNOW

[186 rows x 2 columns]


In [209]:
#On crée une valeur de référence pour discriminer les entreprises en se basant sur la moyenne des valeurs liées à ce ratio. 
dfPS['Price/Sales (ttm)'].mean()

13.177741935483873

In [210]:
#On crée une colonne dans laquelle un booléen vérifie que la valeur du ratio des compagnies ne doit pas excéder 13.  
dfPS['valeurPS<13']= np.where(dfPS['Price/Sales (ttm)']<13, True, False)
dfPS

,Price/Sales (ttm),Symbol,valeurPS<13
150,0.21,SNX,True
172,0.28,ARW,True
178,0.34,FLEX,True
162,0.34,JBL,True
57,0.43,DELL,True
...,...,...,...
42,60.91,DDOG,False
83,71.71,BILL,False
55,73.49,NET,False
127,96.85,S,False


Quatrième sous data frame avec le ratio : Enterprise Value/Revenue

In [211]:
#Ce ratio est l'indicateur qui permet de définir la valeur d'une entreprise en lien avec ses revenus 
# Plus ce ratio est bas, plus l'entreprise est sous évaluée ce qui veut dire qu'il est bon d'investir dans son activité. Plus il est haut, plus cela est synonyme de surestimation de la valeur de l'entreprise.  '

dfEVR=data[['Enterprise Value/Revenue ', 'Symbol']]
dfEVR = dfEVR[~dfEVR['Enterprise Value/Revenue '].isna()]
dfEVR=dfEVR.sort_values(by=[ 'Enterprise Value/Revenue ', 'Symbol'], ascending=True)
print(dfEVR)

     Enterprise Value/Revenue  Symbol
77                        0.01    CAJ
18                        0.02   SONY
114                       0.05    ASX
44                        0.06    WIT
74                        0.13    UMC
..                         ...    ...
42                       61.13   DDOG
127                      69.19      S
55                       75.81    NET
83                       81.62   BILL
22                      107.23   SNOW

[186 rows x 2 columns]


In [212]:
#On crée une valeur de référence à partir des données liées à ce ratio
dfEVR['Enterprise Value/Revenue '].mean()

12.978279569892473

In [213]:
#Création d'une colonne formée de booléens où la condition doit être vérifiée 
dfEVR['valeurEVR<13']= np.where(dfEVR['Enterprise Value/Revenue ']<13, True, False)
dfEVR

,Enterprise Value/Revenue,Symbol,valeurEVR<13
77,0.01,CAJ,True
18,0.02,SONY,True
114,0.05,ASX,True
44,0.06,WIT,True
74,0.13,UMC,True
...,...,...,...
42,61.13,DDOG,False
127,69.19,S,False
55,75.81,NET,False
83,81.62,BILL,False


Cinquième sous data frame avec le ratio : Enterprise Value/EBITDA

In [214]:
#Enterprise Value/EBITDA (EV/EBITDA) est le ratio qui compare les revenus d'une entreprise à son EBITDA
#un petit EV/EBITDA signifie que la compagnie peut être sous évaluée alors qu'un grand EV/EBITDA signifie que la compagnie peut être surévaluée.
#Selon les spécialistes financiers, un ratio EV/EBITDA en-dessous de 10 est un bon signe. La moyenne de ce ratio EV/EBITDA pour la totalité du marché est 14.7.


dfEVEBITDA=data[['Enterprise Value/EBITDA ', 'Symbol']]
dfEVEBITDA = dfEVEBITDA[~dfEVEBITDA['Enterprise Value/EBITDA '].isna()]
dfEVEBITDA=dfEVEBITDA.sort_values(by=[ 'Enterprise Value/EBITDA ', 'Symbol'], ascending=True)
print(dfEVEBITDA)

dfEVEBITDA['Enterprise Value/EBITDA '].mean()

     Enterprise Value/EBITDA  Symbol
49                    -654.11   CRWD
55                    -374.85    NET
173                   -255.86   PEGA
53                    -211.68     ZS
103                   -197.56    RNG
..                        ...    ...
158                    133.89    DLO
157                    134.27   DAVA
85                     143.80     ZI
31                     175.81   WDAY
20                     183.23    NOW

[186 rows x 2 columns]


6.2202688172043015

In [215]:
# la moyenne = 6.2 <10. On chosit de prendre pour valeur référente 10 d'après ci-dessus.
dfEVEBITDA['valeurEVEBITDA<10']=np.where(dfEVEBITDA['Enterprise Value/EBITDA ']<10, True, False)
dfEVEBITDA


,Enterprise Value/EBITDA,Symbol,valeurEVEBITDA<10
49,-654.11,CRWD,True
55,-374.85,NET,True
173,-255.86,PEGA,True
53,-211.68,ZS,True
103,-197.56,RNG,True
...,...,...,...
158,133.89,DLO,False
157,134.27,DAVA,False
85,143.80,ZI,False
31,175.81,WDAY,False


Sixième sous data frame avec le ratio : Profit Margin

In [216]:
#Profit Margin sert à mesure la performance financière de l'entreprise
#Plus le Profit Margin (PM), mieux c'est.
#Par exemple : l'actuel PM de Microsoft est de 19.57%, ie que l'entreprise a un bénéfice net de 0.19$ pour chaque dollar de son revenu global généré . 


dfPMargin=data[['Profit Margin ', 'Symbol']]
dfPMargin = dfPMargin[~dfPMargin['Profit Margin '].isna()]
dfPMargin=dfPMargin.sort_values(by=[ 'Profit Margin ', 'Symbol'], ascending=True)
print(dfPMargin)

dfPMargin['Profit Margin '].mean()

     Profit Margin  Symbol
383         -2.2030     DM
386         -1.9251     EB
118         -1.8363   HOOD
322         -1.8023   STEM
447         -1.6396   DMRC
..              ...    ...
485          0.5302   IMMR
241          0.6182    CDK
439          0.7089     MX
15           0.8100   SHOP
181          2.0599   SLAB

[482 rows x 2 columns]


-0.03657593360995839

In [217]:
dfPMargin['valeurPM>-0.03']=np.where(dfPMargin['Profit Margin ']>-0.03, True, False)
dfPMargin

,Profit Margin,Symbol,valeurPM>-0.03
383,-2.2030,DM,False
386,-1.9251,EB,False
118,-1.8363,HOOD,False
322,-1.8023,STEM,False
447,-1.6396,DMRC,False
...,...,...,...
485,0.5302,IMMR,True
241,0.6182,CDK,True
439,0.7089,MX,True
15,0.8100,SHOP,True


Septième sous data frame avec le ratio : Operating Margin

In [218]:
#Operating Margin measure l'efficience de la stratégie de pricing d'une entreprise.
#Plus l'Operating Margin est grand, plus cela est synonyme de bon signe. 

dfOpMargin=data[['Operating Margin (ttm)', 'Symbol']]
dfOpMargin = dfOpMargin[~dfOpMargin['Operating Margin (ttm)'].isna()]
dfOpMargin=dfOpMargin.sort_values(by=[ 'Operating Margin (ttm)', 'Symbol'], ascending=True)
print(dfOpMargin)
dfOpMargin['Operating Margin (ttm)'].mean()


     Operating Margin (ttm) Symbol
121                -10.2211    AUR
214                 -7.8969   LAZR
450                 -7.4728   RKLY
463                 -4.4498   BTCM
423                 -3.4413   LTCH
..                      ...    ...
485                  0.4608   IMMR
14                   0.4704    TXN
152                  0.5085   AZPN
34                   0.5410   COIN
76                   0.6495   VRSN

[482 rows x 2 columns]


-0.08111120331950239

In [219]:
dfOpMargin['valeurOpM>-0.08']=np.where(dfOpMargin['Operating Margin (ttm)']>-0.08, True, False)
dfOpMargin

,Operating Margin (ttm),Symbol,valeurOpM>-0.08
121,-10.2211,AUR,False
214,-7.8969,LAZR,False
450,-7.4728,RKLY,False
463,-4.4498,BTCM,False
423,-3.4413,LTCH,False
...,...,...,...
485,0.4608,IMMR,True
14,0.4704,TXN,True
152,0.5085,AZPN,True
34,0.5410,COIN,True


Huitième sous data frame avec le ratio : Return on Assets (ROA)

In [220]:
#Return on Assets sert à décrire l'efficacité de la politique manageriale pour tirer bénéfice des avantages d'une entreprise .
# Plus le ROA est grand, plus c'est bon signe d'investir dedans.

dfROA=data[['Return on Assets (ttm)', 'Symbol']]
dfROA = dfROA[~dfROA['Return on Assets (ttm)'].isna()]
dfROA=dfROA.sort_values(by=[ 'Return on Assets (ttm)', 'Symbol'], ascending=True)
print(dfROA)
dfROA['Return on Assets (ttm)'].mean()


     Return on Assets (ttm) Symbol
97                  -0.4405     XM
460                 -0.3304   LVOX
499                 -0.3304  LVOXU
447                 -0.3034   DMRC
214                 -0.2721   LAZR
..                      ...    ...
227                  0.2198   CRCT
347                  0.2374   HIMX
14                   0.2493    TXN
76                   0.2966   VRSN
100                  0.5101     UI

[409 rows x 2 columns]


0.027006601466992675

In [221]:
dfROA['valeurROA>0.02']=np.where(dfROA['Return on Assets (ttm)']>0.02, True, False)
dfROA

,Return on Assets (ttm),Symbol,valeurROA>0.02
97,-0.4405,XM,False
460,-0.3304,LVOX,False
499,-0.3304,LVOXU,False
447,-0.3034,DMRC,False
214,-0.2721,LAZR,False
...,...,...,...
227,0.2198,CRCT,True
347,0.2374,HIMX,True
14,0.2493,TXN,True
76,0.2966,VRSN,True


Neuvième sous data frame avec le ratio : Return on Equity

In [222]:
#Return on Equity est le ratio qui montre combien l'entreprise est bénéfique pour l'actionnaire. 
#Plus la valeur du ratio est haute plus c'est un bon signe

dfROE=data[['Return on Equity (ttm)', 'Symbol']]
dfROE = dfROE[~dfROE['Return on Equity (ttm)'].isna()]
dfROE=dfROE.sort_values(by=[ 'Return on Equity (ttm)', 'Symbol'], ascending=True)
print(dfROE)
dfROE['Return on Equity (ttm)'].mean() #=0.02
dfROE['valeurROE>0.02']=np.where(dfROE['Return on Equity (ttm)']>0.02, True, False)
dfROE

     Return on Equity (ttm) Symbol
453                 -8.3507   INSG
326                 -3.7077   SABR
27                  -3.2512   TEAM
500                 -1.9451  SABRP
119                 -1.8884   ASAN
..                      ...    ...
96                   1.6687   NTAP
35                   1.9555   ADSK
470                  2.1114   MAPS
284                  2.2567   SPWR
142                  3.5621   FICO

[396 rows x 2 columns]


,Return on Equity (ttm),Symbol,valeurROE>0.02
453,-8.3507,INSG,False
326,-3.7077,SABR,False
27,-3.2512,TEAM,False
500,-1.9451,SABRP,False
119,-1.8884,ASAN,False
...,...,...,...
96,1.6687,NTAP,True
35,1.9555,ADSK,True
470,2.1114,MAPS,True
284,2.2567,SPWR,True


Dixième sous data frame avec le ratio : Revenue

In [223]:
#Revenue (ttm): est le ratio qui recoupe tous les bénéfices engendrés par l'entreprise au cours d'une année. 
#Plus la valeur associée est haute plus cela est bon signe.

data['Revenue (ttm)'] = np.where(data['Revenue (ttm)'].str.contains("M"),data['Revenue (ttm)'].map(lambda x : 1000000*float(str(x).replace("M","")) if ("M" in str(x)) else x),data['Revenue (ttm)'])
data['Revenue (ttm)'] = np.where(data['Revenue (ttm)'].str.contains("B"),data['Revenue (ttm)'].map(lambda x : 1000000000*float(str(x).replace("B","")) if ("B" in str(x)) else x),data['Revenue (ttm)'])
data['Revenue (ttm)'] = np.where(data['Revenue (ttm)'].str.contains("T"),data['Revenue (ttm)'].map(lambda x : 1000000000000*float(str(x).replace("T","")) if ("T" in str(x)) else x),data['Revenue (ttm)'])
dfRevenue=data[['Revenue (ttm)', 'Symbol']]
dfRevenue = dfRevenue[~dfRevenue['Revenue (ttm)'].isna()]
dfRevenue=dfRevenue.sort_values(by=[ 'Revenue (ttm)', 'Symbol'], ascending=True)
print(dfRevenue)

    Revenue (ttm) Symbol
450     1.177e+07   RKLY
395     1.733e+07    VLD
338     1.968e+07    WBX
423     2.196e+07   LTCH
214     2.203e+07   LAZR
..            ...    ...
44     6.9835e+11    WIT
3        1.51e+12    TSM
77        3.5e+12    CAJ
18       9.56e+12   SONY
194     2.805e+13    LPL

[477 rows x 2 columns]


In [224]:
dfRevenue['Revenue>97945222473']=np.where(dfRevenue['Revenue (ttm)']>97945222473, True, False)
dfRevenue

,Revenue (ttm),Symbol,Revenue>97945222473
450,1.177e+07,RKLY,False
395,1.733e+07,VLD,False
338,1.968e+07,WBX,False
423,2.196e+07,LTCH,False
214,2.203e+07,LAZR,False
...,...,...,...
44,6.9835e+11,WIT,True
3,1.51e+12,TSM,True
77,3.5e+12,CAJ,True
18,9.56e+12,SONY,True


Onzième sous data frame avec le ratio : Revenue Per Share

In [225]:
#Revenue Per Share (ttm): est le ratio qui sert de comparaison entre les revenus d'une entreprises et ses parts . 
#Plus ce ratio est haut plus c'est bon signe d'investir.


dfRevShare=data[['Revenue Per Share (ttm)', 'Symbol']]
dfRevShare = dfRevShare[~dfRevShare['Revenue Per Share (ttm)'].isna()]
dfRevShare=dfRevShare.sort_values(by=[ 'Revenue Per Share (ttm)', 'Symbol'], ascending=True)
print(dfRevShare)
dfRevShare['Revenue Per Share (ttm)'].mean()

     Revenue Per Share (ttm) Symbol
462                     0.04   BKKT
214                     0.07   LAZR
383                     0.20     DM
121                     0.26    AUR
409                     0.28     GB
..                       ...    ...
220                  1408.53   SRAD
411                  1658.71   ENFN
77                   3350.34    CAJ
18                   7719.09   SONY
194                 39192.22    LPL

[475 rows x 2 columns]


142.14355789473686

In [226]:
dfRevShare['valeurRevShare>142']=np.where(dfRevShare['Revenue Per Share (ttm)']>142, True, False)
dfRevShare

,Revenue Per Share (ttm),Symbol,valeurRevShare>142
462,0.04,BKKT,False
214,0.07,LAZR,False
383,0.20,DM,False
121,0.26,AUR,False
409,0.28,GB,False
...,...,...,...
220,1408.53,SRAD,True
411,1658.71,ENFN,True
77,3350.34,CAJ,True
18,7719.09,SONY,True


Douzième sous data frame avec le ratio : Gross Profit

In [227]:
#Gross Profit (ttm) est l'ensemble des revenus d'une années moins les prix des biens vendus. Il sert à décrire la productivité laborale d'une entreprise. 
#Plus ce ratio est haut, mieux c'est.

data['Gross Profit (ttm)'] = np.where(data['Gross Profit (ttm)'].str.contains("k"),data['Gross Profit (ttm)'].map(lambda x : 1000*float(str(x).replace("k","")) if ("k" in str(x)) else x),data['Gross Profit (ttm)'])
data['Gross Profit (ttm)'] = np.where(data['Gross Profit (ttm)'].str.contains("M"),data['Gross Profit (ttm)'].map(lambda x : 1000000*float(str(x).replace("M","")) if ("M" in str(x)) else x),data['Gross Profit (ttm)'])
data['Gross Profit (ttm)'] = np.where(data['Gross Profit (ttm)'].str.contains("B"),data['Gross Profit (ttm)'].map(lambda x : 1000000000*float(str(x).replace("B","")) if ("B" in str(x)) else x),data['Gross Profit (ttm)'])
data['Gross Profit (ttm)'] = np.where(data['Gross Profit (ttm)'].str.contains("T"),data['Gross Profit (ttm)'].map(lambda x : 1000000000000*float(str(x).replace("T","")) if ("T" in str(x)) else x),data['Gross Profit (ttm)'])
dfGProfit=data[['Gross Profit (ttm)', 'Symbol']]
dfGProfit = dfGProfit[~dfGProfit['Gross Profit (ttm)'].isna()]
dfGProfit=dfGProfit.sort_values(by=[ 'Gross Profit (ttm)', 'Symbol'], ascending=True)
print(dfGProfit)
dfGProfit['Gross Profit (ttm)'].mean() #=18117636428


    Gross Profit (ttm) Symbol
68         -7.1272e+08    GFS
462         -3.014e+07   BKKT
383         -1.505e+07     DM
214           -1.1e+07   LAZR
451          -9.78e+06   MAXN
..                 ...    ...
44          1.9622e+11    WIT
3           7.1115e+11    TSM
77            1.38e+12    CAJ
18            2.43e+12   SONY
194           2.64e+12    LPL

[476 rows x 2 columns]


18117636428.57143

In [228]:
dfGProfit['GProfit>18117636428 ']=np.where(dfGProfit['Gross Profit (ttm)']>18117636428, True, False)
dfGProfit

,Gross Profit (ttm),Symbol,GProfit>18117636428
68,-7.1272e+08,GFS,False
462,-3.014e+07,BKKT,False
383,-1.505e+07,DM,False
214,-1.1e+07,LAZR,False
451,-9.78e+06,MAXN,False
...,...,...,...
44,1.9622e+11,WIT,True
3,7.1115e+11,TSM,True
77,1.38e+12,CAJ,True
18,2.43e+12,SONY,True


Treizième sous data frame avec le ratio : EBIDTDA

In [229]:
#EBITDA (earnings before interest, tax, depreciation and amortization) est le revenu net d'une entreprise une fois l'ensemble des taxes, dévalutations etc prise en compte . 
#Plus c'est grand mieux c'est.


data['EBITDA '] = np.where(data['EBITDA '].str.contains("k"),data['EBITDA '].map(lambda x : 1000*float(str(x).replace("k","")) if ("k" in str(x)) else x),data['EBITDA '])
data['EBITDA '] = np.where(data['EBITDA '].str.contains("M"),data['EBITDA '].map(lambda x : 1000000*float(str(x).replace("M","")) if ("M" in str(x)) else x),data['EBITDA '])
data['EBITDA '] = np.where(data['EBITDA '].str.contains("B"),data['EBITDA '].map(lambda x : 1000000000*float(str(x).replace("B","")) if ("B" in str(x)) else x),data['EBITDA '])
data['EBITDA '] = np.where(data['EBITDA '].str.contains("T"),data['EBITDA '].map(lambda x : 1000000000000*float(str(x).replace("T","")) if ("T" in str(x)) else x),data['EBITDA '])
dfEBITDA=data[['EBITDA ', 'Symbol']]
dfEBITDA = dfEBITDA[~dfEBITDA['EBITDA '].isna()]
dfEBITDA=dfEBITDA.sort_values(by=[ 'EBITDA ', 'Symbol'], ascending=True)
print(dfEBITDA)
dfEBITDA['EBITDA '].mean() #=21000376527
dfEBITDA['EBITDA>21000376527 ']=np.where(dfEBITDA['EBITDA ']>21000376527, True, False)
dfEBITDA


        EBITDA  Symbol
73   -1.141e+10   DIDI
153   -5.16e+09    YMM
28    -2.73e+09   UBER
129   -1.13e+09   LYFT
106  -9.874e+08   SPLK
..          ...    ...
44   1.5743e+11    WIT
77   5.2253e+11    CAJ
3      1.03e+12    TSM
18     1.47e+12   SONY
194    6.03e+12    LPL

[476 rows x 2 columns]


,EBITDA,Symbol,EBITDA>21000376527
73,-1.141e+10,DIDI,False
153,-5.16e+09,YMM,False
28,-2.73e+09,UBER,False
129,-1.13e+09,LYFT,False
106,-9.874e+08,SPLK,False
...,...,...,...
44,1.5743e+11,WIT,True
77,5.2253e+11,CAJ,True
3,1.03e+12,TSM,True
18,1.47e+12,SONY,True


Quatorzième sous data frame avec le ratio : Total Cash Per Share (TCPS)

In [230]:
#un plus gros TCPS montre que l'entreprise peut faire face aux aléas financiers 
dfTCPS=data[['Total Cash Per Share (mrq)', 'Symbol']]
dfTCPS = dfTCPS[~dfTCPS['Total Cash Per Share (mrq)'].isna()]
dfTCPS=dfTCPS.sort_values(by=[ 'Total Cash Per Share (mrq)', 'Symbol'], ascending=True)
dfTCPS['Total Cash Per Share (mrq)'].mean() #=24.5
dfTCPS['valeurTCPS>24.5']=np.where(dfTCPS['Total Cash Per Share (mrq)']>24.5, True, False)
dfTCPS




,Total Cash Per Share (mrq),Symbol,valeurTCPS>24.5
246,0.00,ALIT,False
462,0.06,BKKT,False
235,0.09,DV,False
486,0.09,LYTS,False
467,0.11,IIIV,False
...,...,...,...
77,461.22,CAJ,True
220,553.32,SRAD,True
466,616.13,QIWI,True
18,705.38,SONY,True


Quinzième sous data frame avec le ratio : Levered Free Cash Flow (LFCF)

In [231]:
#Un LFCF positif montre la capacité de réponse de l'entreprise face aux aléas financiers 

data['Levered Free Cash Flow (ttm)'] = np.where(data['Levered Free Cash Flow (ttm)'].str.contains("k"),data['Levered Free Cash Flow (ttm)'].map(lambda x : 1000*float(str(x).replace("k","")) if ("k" in str(x)) else x),data['Levered Free Cash Flow (ttm)'])
data['Levered Free Cash Flow (ttm)'] = np.where(data['Levered Free Cash Flow (ttm)'].str.contains("M"),data['Levered Free Cash Flow (ttm)'].map(lambda x : 1000000*float(str(x).replace("M","")) if ("M" in str(x)) else x),data['Levered Free Cash Flow (ttm)'])
data['Levered Free Cash Flow (ttm)'] = np.where(data['Levered Free Cash Flow (ttm)'].str.contains("B"),data['Levered Free Cash Flow (ttm)'].map(lambda x : 1000000000*float(str(x).replace("B","")) if ("B" in str(x)) else x),data['Levered Free Cash Flow (ttm)'])
data['Levered Free Cash Flow (ttm)'] = np.where(data['Levered Free Cash Flow (ttm)'].str.contains("T"),data['Levered Free Cash Flow (ttm)'].map(lambda x : 1000000000000*float(str(x).replace("T","")) if ("T" in str(x)) else x),data['Levered Free Cash Flow (ttm)'])
dfLFCF=data[['Levered Free Cash Flow (ttm)', 'Symbol']]
dfLFCF = dfLFCF[~dfLFCF['Levered Free Cash Flow (ttm)'].isna()]
dfLFCF=dfLFCF.sort_values(by=['Levered Free Cash Flow (ttm)', 'Symbol'], ascending=True)
dfLFCF

,Levered Free Cash Flow (ttm),Symbol
331,-2.72e+09,CD
290,-2.63e+09,KC
204,-2.62e+09,MSTR
406,-1.72e+09,VNET
195,-1.6e+09,RUN
...,...,...
0,7.33e+10,AAPL
3,1.0362e+11,TSM
77,3.1369e+11,CAJ
194,7.2253e+11,LPL


In [232]:
dfLFCF['valeurLFCF>0']= np.where(dfLFCF['Levered Free Cash Flow (ttm)']>0, True, False)
dfLFCF

,Levered Free Cash Flow (ttm),Symbol,valeurLFCF>0
331,-2.72e+09,CD,False
290,-2.63e+09,KC,False
204,-2.62e+09,MSTR,False
406,-1.72e+09,VNET,False
195,-1.6e+09,RUN,False
...,...,...,...
0,7.33e+10,AAPL,True
3,1.0362e+11,TSM,True
77,3.1369e+11,CAJ,True
194,7.2253e+11,LPL,True


In [233]:
#le current ratio doit être compris entre ]1;3[
dfCR=data[['Current Ratio (mrq)', 'Symbol']]
dfCR=dfCR[~dfCR['Current Ratio (mrq)'].isna()]
dfCR= dfCR.sort_values(by=['Current Ratio (mrq)', 'Symbol'], ascending=True)
dfCR['1<valeurCR<3']= dfCR['Current Ratio (mrq)'].apply(lambda x : True if ((x >1) & (x < 3)) else False)
dfCR

,Current Ratio (mrq),Symbol,1<valeurCR<3
246,0.23,ALIT,False
291,0.32,PWSC,False
242,0.44,MCFE,False
355,0.48,CYXT,False
197,0.52,SWCH,False
...,...,...,...
383,13.49,DM,False
15,13.97,SHOP,False
295,14.26,TUYA,False
214,17.41,LAZR,False


Seizième sous data frame avec le ratio : Total Debt/Equity (TDE) 

In [234]:
#Le TDE doit être en dessous de 200 sinon cela veut dire que l'entreprise recourt à trop de dettes pour s'auto-financer
dfTDE=data[['Total Debt/Equity (mrq)', 'Symbol']]
dfTDE = dfTDE[~dfTDE['Total Debt/Equity (mrq)'].isna()]
dfTDE=dfTDE.sort_values(by=['Total Debt/Equity (mrq)', 'Symbol'], ascending=True)
dfTDE['valeurTDE<200']=np.where(dfTDE['Total Debt/Equity (mrq)']<200, True, False)
dfTDE

,Total Debt/Equity (mrq),Symbol,valeurTDE<200
408,0.05,BTRS,True
320,0.37,ACVA,True
292,0.43,AI,True
89,0.50,MPWR,True
383,0.65,DM,True
...,...,...,...
106,1858.74,SPLK,False
103,3167.78,RNG,False
97,3919.40,XM,False
326,15374.26,SABR,False


Fusion de tous les data frames True/False

In [235]:
#fusion de tous les df True/Flase de chaque critère en fonction de la colonne entreprise
#mettre des NaN si données manquantes sur certains critères pour une entreprise

dfs =dfs=[dfPEfusion, dfPEG, dfPS, dfEVR, dfEVEBITDA, dfPMargin, dfOpMargin, dfROA, dfROE, dfRevenue, dfRevShare, dfGProfit, dfEBITDA, dfTCPS, dfLFCF, dfCR, dfTDE]

# On crée un df vide avec les colonnes et lignes 
cols, rows = [], []
for df_i in dfs:
    cols = cols + df_i.columns.tolist()
    rows = rows + df_i.index.tolist()
cols = np.unique(cols)
rows = np.unique(rows)       
df = pd.DataFrame(data=np.NaN, columns=cols, index=rows) 

# On rempli ce df vierge avec nos df T/F
for df_i in dfs:
    df.loc[df_i.index, df_i.columns] = df_i.values

df

,1<valeurCR<3,Current Ratio (mrq),EBITDA,EBITDA>21000376527,Enterprise Value/EBITDA,Enterprise Value/Revenue,Forward P/E,GProfit>18117636428,Gross Profit (ttm),Levered Free Cash Flow (ttm),...,valeurLFCF>0,valeurOpM>-0.08,valeurPEG<1,valeurPM>-0.03,valeurPS<13,valeurROA>0.02,valeurROE>0.02,valeurRevShare>142,valeurTCPS>24.5,valeurTDE<200
0,True,1.08,1.202300e+11,True,24.41,8.22,32.05,True,1.528400e+11,7.330000e+10,...,True,True,False,True,True,True,True,False,False,False
1,True,2.16,8.575000e+10,True,27.81,14.21,37.45,True,1.158600e+11,4.982000e+10,...,True,True,False,True,False,True,True,False,False,True
2,False,7.14,9.760000e+09,False,75.30,30.79,58.82,False,1.056000e+10,5.940000e+09,...,True,True,False,True,False,True,True,False,False,True
3,True,2.09,1.030000e+12,True,0.53,0.37,25.38,True,7.111500e+11,1.036200e+11,...,True,True,False,True,True,True,True,True,True,True
4,True,1.71,6.600000e+09,False,47.70,17.86,40.82,False,6.800000e+09,6.560000e+09,...,True,True,False,True,False,True,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,False,0.98,1.753000e+07,False,NaN,NaN,NaN,False,1.139600e+08,3.046000e+07,...,True,True,NaN,True,NaN,False,False,False,False,True
499,False,6.31,-8.252000e+07,False,NaN,NaN,NaN,NaN,NaN,1.354000e+07,...,True,False,NaN,False,NaN,False,False,False,False,True
500,False,3.04,-7.893100e+08,False,NaN,NaN,NaN,False,7.740900e+08,-3.313700e+08,...,False,False,NaN,False,NaN,False,False,False,False,False
501,True,2.97,6.831200e+08,False,NaN,NaN,NaN,False,1.220000e+09,2.756700e+08,...,True,True,NaN,True,NaN,True,True,False,False,True


In [236]:
df.columns

Index(['1<valeurCR<3', 'Current Ratio (mrq)', 'EBITDA ', 'EBITDA>21000376527 ',
       'Enterprise Value/EBITDA ', 'Enterprise Value/Revenue ', 'Forward P/E ',
       'GProfit>18117636428 ', 'Gross Profit (ttm)',
       'Levered Free Cash Flow (ttm)', 'Operating Margin (ttm)',
       'PEG Ratio (5 yr expected) ', 'Price/Sales (ttm)', 'Profit Margin ',
       'Return on Assets (ttm)', 'Return on Equity (ttm)', 'Revenue (ttm)',
       'Revenue Per Share (ttm)', 'Revenue>97945222473', 'Symbol',
       'Total Cash Per Share (mrq)', 'Total Debt/Equity (mrq)',
       'Trailing P/E ', 'conditionTrailingPE', 'valeurEVEBITDA<10',
       'valeurEVR<13', 'valeurLFCF>0', 'valeurOpM>-0.08', 'valeurPEG<1',
       'valeurPM>-0.03', 'valeurPS<13', 'valeurROA>0.02', 'valeurROE>0.02',
       'valeurRevShare>142', 'valeurTCPS>24.5', 'valeurTDE<200'],
      dtype='object')

In [237]:
#on veut laisser les entreprises sur la première colonne du df

first_column = df.pop('Symbol') 
  
df.insert(0, 'Symbol', first_column)
df

,Symbol,1<valeurCR<3,Current Ratio (mrq),EBITDA,EBITDA>21000376527,Enterprise Value/EBITDA,Enterprise Value/Revenue,Forward P/E,GProfit>18117636428,Gross Profit (ttm),...,valeurLFCF>0,valeurOpM>-0.08,valeurPEG<1,valeurPM>-0.03,valeurPS<13,valeurROA>0.02,valeurROE>0.02,valeurRevShare>142,valeurTCPS>24.5,valeurTDE<200
0,AAPL,True,1.08,1.202300e+11,True,24.41,8.22,32.05,True,1.528400e+11,...,True,True,False,True,True,True,True,False,False,False
1,MSFT,True,2.16,8.575000e+10,True,27.81,14.21,37.45,True,1.158600e+11,...,True,True,False,True,False,True,True,False,False,True
2,NVDA,False,7.14,9.760000e+09,False,75.30,30.79,58.82,False,1.056000e+10,...,True,True,False,True,False,True,True,False,False,True
3,TSM,True,2.09,1.030000e+12,True,0.53,0.37,25.38,True,7.111500e+11,...,True,True,False,True,True,True,True,True,True,True
4,ASML,True,1.71,6.600000e+09,False,47.70,17.86,40.82,False,6.800000e+09,...,True,True,False,True,False,True,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,SANG,False,0.98,1.753000e+07,False,NaN,NaN,NaN,False,1.139600e+08,...,True,True,NaN,True,NaN,False,False,False,False,True
499,LVOXU,False,6.31,-8.252000e+07,False,NaN,NaN,NaN,NaN,NaN,...,True,False,NaN,False,NaN,False,False,False,False,True
500,SABRP,False,3.04,-7.893100e+08,False,NaN,NaN,NaN,False,7.740900e+08,...,False,False,NaN,False,NaN,False,False,False,False,False
501,IIVIP,True,2.97,6.831200e+08,False,NaN,NaN,NaN,False,1.220000e+09,...,True,True,NaN,True,NaN,True,True,False,False,True


Création d'un df composé uniquement des colonnes entreprise et des df T/F créés au préalables 

In [238]:
#on va éliminer les colonnes autres que celles de nos critères remplies par True/False
# il faut ruser avec quelques colonnes qui ont la valeur 1 ou 0 (qui sont respectivement égales à True ou False) d'où la seconde boucle for
l=[]
for c in df.columns :
    if ( (True in list(df[c])) or(False in list(df[c])) or (c=='Symbol') ):
        l.append(c)        
df=df[l]


l=[]
for c in df.columns :
    if c != 'Symbol' :
        if ( len(df[(df[c]>0) & (df[c]<1)])==0) and ( len(df[(df[c]<0) & (df[c]>1)])==0):
            l.append(c)
    else : 
        l.append(c)
df=df[l]

df

,Symbol,1<valeurCR<3,EBITDA>21000376527,GProfit>18117636428,Revenue>97945222473,conditionTrailingPE,valeurEVEBITDA<10,valeurEVR<13,valeurLFCF>0,valeurOpM>-0.08,valeurPEG<1,valeurPM>-0.03,valeurPS<13,valeurROA>0.02,valeurROE>0.02,valeurRevShare>142,valeurTCPS>24.5,valeurTDE<200
0,AAPL,True,True,True,True,False,False,True,True,True,False,True,True,True,True,False,False,False
1,MSFT,True,True,True,True,True,False,False,True,True,False,True,False,True,True,False,False,True
2,NVDA,False,False,False,False,False,False,False,True,True,False,True,False,True,True,False,False,True
3,TSM,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True
4,ASML,True,False,False,False,True,False,False,True,True,False,True,False,True,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,SANG,False,False,False,False,NaN,NaN,NaN,True,True,NaN,True,NaN,False,False,False,False,True
499,LVOXU,False,False,NaN,False,NaN,NaN,NaN,True,False,NaN,False,NaN,False,False,False,False,True
500,SABRP,False,False,False,False,NaN,NaN,NaN,False,False,NaN,False,NaN,False,False,False,False,False
501,IIVIP,True,False,False,False,NaN,NaN,NaN,True,True,NaN,True,NaN,True,True,False,False,True


In [239]:
# on remplace les nan par une valeur très proche de 0 de sorte à ce qu'elle n'affecte pas le scoring à venir. 
#On ne met pas 0 à cause du False : cela gêne les manip à venir.
df= df.fillna(0.00000000000001)
df

,Symbol,1<valeurCR<3,EBITDA>21000376527,GProfit>18117636428,Revenue>97945222473,conditionTrailingPE,valeurEVEBITDA<10,valeurEVR<13,valeurLFCF>0,valeurOpM>-0.08,valeurPEG<1,valeurPM>-0.03,valeurPS<13,valeurROA>0.02,valeurROE>0.02,valeurRevShare>142,valeurTCPS>24.5,valeurTDE<200
0,AAPL,True,True,True,True,False,False,True,True,True,False,True,True,True,True,False,False,False
1,MSFT,True,True,True,True,True,False,False,True,True,False,True,False,True,True,False,False,True
2,NVDA,False,False,False,False,False,False,False,True,True,False,True,False,True,True,False,False,True
3,TSM,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True
4,ASML,True,False,False,False,True,False,False,True,True,False,True,False,True,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,SANG,False,False,False,False,1e-14,1e-14,1e-14,True,True,1e-14,True,1e-14,False,False,False,False,True
499,LVOXU,False,False,1e-14,False,1e-14,1e-14,1e-14,True,False,1e-14,False,1e-14,False,False,False,False,True
500,SABRP,False,False,False,False,1e-14,1e-14,1e-14,False,False,1e-14,False,1e-14,False,False,False,False,False
501,IIVIP,True,False,False,False,1e-14,1e-14,1e-14,True,True,1e-14,True,1e-14,True,True,False,False,True


# Création de scoring afin d'évaluer les critères d'investissement des entreprises

In [240]:
#construction d'un scoring pondéré en 2 grands axes 
# nos critères étudiés se regroupent en 2 grandes catégories : les "valuation measures" et les "financial highlights"

#Parmi nos critères sélectionnés
#valuation measures : Trailing P/E, PEG Ratio, Price/sales, Enterprise Value/Revenue, Enterprise Value/EBITDA
#financial highlights : Profit Margin, Operating Margin, ROA, ROE, Revenue, Revenue Per Share, Gross Profit, EBITDA, Total Cash per Share, Total Debt/Equity, Current ratio, Levered Free Cash Flow

#On sous divise les financial highlights en 5 sous-catégories :
#Profitability : Profit Margin, Operating Margin
#Management Effectiveness : ROA, ROE
#Income Statement : Revenue, Revenue Per Share, Gross Profit, EBITDA
#Balance Sheet : Total Debt/Equity, Current ratio, Total Cash Per Share
#Cash Flow Statement : Levered Free Cash Flow

#on accorde un poids égal aux financial highlights et aux valuation measures =1
#On pondère chaque sous-partie avec un poids égal dans les financial highlights. Elles ont donc un poids d'1/5
# score max de 2
# Pour les False on met l'opposé du poids donné aux True
# score min de -2



In [241]:
#Construction du scoring pour la partie "Valuation measures"

#il y a 5 critères dans cette partie
#on choisit d'accorder plus d'importance à EV/EBITDA qu'à notre condition Trailing PE conformément à la presse spécialisée dans le domaine
#on accorde pour Trailing P/E, PEG Ratio, Price/sales, Enterprise Value/Revenue, Enterprise Value/EBITDA respectivement un poids de : 1/15, 1/5, 1/5, 1/5, 2/15

l=[ 'valeurPEG<1', 'valeurPS<13','valeurEVR<13']
for c in df[l] :
    if  (True in list(df[c])):
        df[c] = df[c].replace(True,1/5)

    if (False in list(df[c])) :
            df[c] = df[c].replace(False,-1/5)


    
if (True in df['conditionTrailingPE'] ) :
    df['conditionTrailingPE'] = df['conditionTrailingPE'].replace(True, 1/15)
if (False in df['conditionTrailingPE'] ) :
    df['conditionTrailingPE'] = df['conditionTrailingPE'].replace(False, -1/15)

if (True in df['valeurEVEBITDA<10'] ) :
    df['valeurEVEBITDA<10'] = df['valeurEVEBITDA<10'].replace(True, 2/15)
if (False in df['valeurEVEBITDA<10'] ) :
    df['valeurEVEBITDA<10'] = df['valeurEVEBITDA<10'].replace(False, -2/15)
    
    

df


,Symbol,1<valeurCR<3,EBITDA>21000376527,GProfit>18117636428,Revenue>97945222473,conditionTrailingPE,valeurEVEBITDA<10,valeurEVR<13,valeurLFCF>0,valeurOpM>-0.08,valeurPEG<1,valeurPM>-0.03,valeurPS<13,valeurROA>0.02,valeurROE>0.02,valeurRevShare>142,valeurTCPS>24.5,valeurTDE<200
0,AAPL,True,True,True,True,-6.666667e-02,-1.333333e-01,2.000000e-01,True,True,-2.000000e-01,True,2.000000e-01,True,True,False,False,False
1,MSFT,True,True,True,True,6.666667e-02,-1.333333e-01,-2.000000e-01,True,True,-2.000000e-01,True,-2.000000e-01,True,True,False,False,True
2,NVDA,False,False,False,False,-6.666667e-02,-1.333333e-01,-2.000000e-01,True,True,-2.000000e-01,True,-2.000000e-01,True,True,False,False,True
3,TSM,True,True,True,True,6.666667e-02,1.333333e-01,2.000000e-01,True,True,-2.000000e-01,True,2.000000e-01,True,True,True,True,True
4,ASML,True,False,False,False,6.666667e-02,-1.333333e-01,-2.000000e-01,True,True,-2.000000e-01,True,-2.000000e-01,True,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,SANG,False,False,False,False,1.000000e-14,1.000000e-14,1.000000e-14,True,True,1.000000e-14,True,1.000000e-14,False,False,False,False,True
499,LVOXU,False,False,1e-14,False,1.000000e-14,1.000000e-14,1.000000e-14,True,False,1.000000e-14,False,1.000000e-14,False,False,False,False,True
500,SABRP,False,False,False,False,1.000000e-14,1.000000e-14,1.000000e-14,False,False,1.000000e-14,False,1.000000e-14,False,False,False,False,False
501,IIVIP,True,False,False,False,1.000000e-14,1.000000e-14,1.000000e-14,True,True,1.000000e-14,True,1.000000e-14,True,True,False,False,True


In [242]:
#Construction du scoring pour la partie "Financial Highlights"
# 5 sous-parties qui pèsent chacune 1/5
# Leur poids :
#Profitability : Profit Margin, Operating Margin =  1/10 chacun
#Management Effectiveness : ROA, ROE = 1/10 chacun
#Income Statement : Revenue, Revenue Per Share, Gross Profit, EBITDA = 1/20 chacun
#Balance Sheet : Total Debt/Equity, Current ratio, Total Cash Per Share =1/15 chacun
#Cash Flow Statement : Levered Free Cash Flow =1/5

l=[ 'valeurPM>-0.03','valeurOpM>-0.08', 'valeurROA>0.02', 'valeurROE>0.02' ]
for c in df[l] :
    if  (True in list(df[c])):
        df[c] = df[c].replace(True,1/10)

    if (False in list(df[c])) :
            df[c] = df[c].replace(False,-1/10)
            
            
            
l=['valeurTDE<200','1<valeurCR<3', 'valeurTCPS>24.5']
for c in df[l] :
    if (True in list(df[c])):
        df[c] = df[c].replace(True,1/15)

    if (False in list(df[c])) :
            df[c] = df[c].replace(False,-1/15)


l=['Revenue>97945222473', 'valeurRevShare>142', 'GProfit>18117636428 ', 'EBITDA>21000376527 ' ]
for c in df[l] :
    if  (True in list(df[c])):
        df[c] = df[c].replace(True,1/20)

    if (False in list(df[c])) :
            df[c] = df[c].replace(False,-1/20)            

if (True in df['valeurLFCF>0'] ) :
    df['valeurLFCF>0'] = df['valeurLFCF>0'].replace(True, 1/5)
if (False in df['valeurLFCF>0'] ) :
    df['valeurLFCF>0'] = df['valeurLFCF>0'].replace(False, -1/5)
    
df


,Symbol,1<valeurCR<3,EBITDA>21000376527,GProfit>18117636428,Revenue>97945222473,conditionTrailingPE,valeurEVEBITDA<10,valeurEVR<13,valeurLFCF>0,valeurOpM>-0.08,valeurPEG<1,valeurPM>-0.03,valeurPS<13,valeurROA>0.02,valeurROE>0.02,valeurRevShare>142,valeurTCPS>24.5,valeurTDE<200
0,AAPL,0.066667,0.05,5.000000e-02,0.05,-6.666667e-02,-1.333333e-01,2.000000e-01,0.2,0.1,-2.000000e-01,0.1,2.000000e-01,0.1,0.1,-0.05,-0.066667,-0.066667
1,MSFT,0.066667,0.05,5.000000e-02,0.05,6.666667e-02,-1.333333e-01,-2.000000e-01,0.2,0.1,-2.000000e-01,0.1,-2.000000e-01,0.1,0.1,-0.05,-0.066667,0.066667
2,NVDA,-0.066667,-0.05,-5.000000e-02,-0.05,-6.666667e-02,-1.333333e-01,-2.000000e-01,0.2,0.1,-2.000000e-01,0.1,-2.000000e-01,0.1,0.1,-0.05,-0.066667,0.066667
3,TSM,0.066667,0.05,5.000000e-02,0.05,6.666667e-02,1.333333e-01,2.000000e-01,0.2,0.1,-2.000000e-01,0.1,2.000000e-01,0.1,0.1,0.05,0.066667,0.066667
4,ASML,0.066667,-0.05,-5.000000e-02,-0.05,6.666667e-02,-1.333333e-01,-2.000000e-01,0.2,0.1,-2.000000e-01,0.1,-2.000000e-01,0.1,0.1,-0.05,-0.066667,0.066667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,SANG,-0.066667,-0.05,-5.000000e-02,-0.05,1.000000e-14,1.000000e-14,1.000000e-14,0.2,0.1,1.000000e-14,0.1,1.000000e-14,-0.1,-0.1,-0.05,-0.066667,0.066667
499,LVOXU,-0.066667,-0.05,1.000000e-14,-0.05,1.000000e-14,1.000000e-14,1.000000e-14,0.2,0.0,1.000000e-14,0.0,1.000000e-14,-0.1,-0.1,-0.05,-0.066667,0.066667
500,SABRP,-0.066667,-0.05,-5.000000e-02,-0.05,1.000000e-14,1.000000e-14,1.000000e-14,-0.2,0.0,1.000000e-14,0.0,1.000000e-14,-0.1,-0.1,-0.05,-0.066667,-0.066667
501,IIVIP,0.066667,-0.05,-5.000000e-02,-0.05,1.000000e-14,1.000000e-14,1.000000e-14,0.2,0.1,1.000000e-14,0.1,1.000000e-14,0.1,0.1,-0.05,-0.066667,0.066667


In [243]:
l_valuationmeasures=['valeurPEG<1', 'valeurPS<13','valeurEVR<13','conditionTrailingPE','valeurEVEBITDA<10']
df['scoreValuationMeasures']=df[l_valuationmeasures].sum(axis=1)

In [244]:
#on vérifie bien que l'on a des valeurs entre -1 et 1
df=df.sort_values(by=[ 'scoreValuationMeasures', 'Symbol'], ascending=False)
df


,Symbol,1<valeurCR<3,EBITDA>21000376527,GProfit>18117636428,Revenue>97945222473,conditionTrailingPE,valeurEVEBITDA<10,valeurEVR<13,valeurLFCF>0,valeurOpM>-0.08,valeurPEG<1,valeurPM>-0.03,valeurPS<13,valeurROA>0.02,valeurROE>0.02,valeurRevShare>142,valeurTCPS>24.5,valeurTDE<200,scoreValuationMeasures
74,UMC,0.066667,0.05,0.05,0.05,6.666667e-02,0.133333,0.2,0.2,0.1,0.2,0.1,0.2,0.1,0.1,-0.05,0.066667,0.066667,0.800000
70,GLW,0.066667,-0.05,-0.05,-0.05,6.666667e-02,0.133333,0.2,0.2,0.1,0.2,0.1,0.2,0.1,0.1,-0.05,-0.066667,0.066667,0.800000
107,WDC,0.066667,-0.05,-0.05,-0.05,1.000000e-14,0.133333,0.2,0.2,0.1,0.2,0.1,0.2,0.1,0.1,-0.05,-0.066667,0.066667,0.733333
150,SNX,0.066667,-0.05,-0.05,-0.05,1.000000e-14,0.133333,0.2,0.2,0.1,0.2,0.1,0.2,0.1,0.1,0.05,0.066667,0.066667,0.733333
172,ARW,0.066667,-0.05,-0.05,-0.05,1.000000e-14,0.133333,0.2,0.2,0.1,0.2,0.1,0.2,0.1,0.1,0.05,-0.066667,0.066667,0.733333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,MPWR,-0.066667,-0.05,-0.05,-0.05,-6.666667e-02,-0.133333,-0.2,0.2,0.1,-0.2,0.1,-0.2,0.1,0.1,-0.05,-0.066667,0.066667,-0.800000
13,INTU,0.066667,-0.05,-0.05,-0.05,-6.666667e-02,-0.133333,-0.2,0.2,0.1,-0.2,0.1,-0.2,0.1,0.1,-0.05,-0.066667,0.066667,-0.800000
69,HUBS,0.066667,-0.05,-0.05,-0.05,-6.666667e-02,-0.133333,-0.2,0.2,0.1,-0.2,0.0,-0.2,-0.1,-0.1,-0.05,0.066667,0.066667,-0.800000
45,CDNS,0.066667,-0.05,-0.05,-0.05,-6.666667e-02,-0.133333,-0.2,0.2,0.1,-0.2,0.1,-0.2,0.1,0.1,-0.05,-0.066667,0.066667,-0.800000


In [245]:
l_financialhighlights=['valeurPM>-0.03','valeurOpM>-0.08', 'valeurROA>0.02', 'valeurROE>0.02','valeurTDE<200','1<valeurCR<3', 'valeurTCPS>24.5', 'Revenue>97945222473', 'valeurRevShare>142', 'GProfit>18117636428 ', 'EBITDA>21000376527 ','valeurLFCF>0']
df['scoreFinancialHighlights']=df[l_financialhighlights].sum(axis=1)

In [246]:
#on vérifie bien que l'on a des valeurs entre -1 et 1
df=df.sort_values(by=[ 'scoreFinancialHighlights', 'Symbol'], ascending=False)
df


,Symbol,1<valeurCR<3,EBITDA>21000376527,GProfit>18117636428,Revenue>97945222473,conditionTrailingPE,valeurEVEBITDA<10,valeurEVR<13,valeurLFCF>0,valeurOpM>-0.08,valeurPEG<1,valeurPM>-0.03,valeurPS<13,valeurROA>0.02,valeurROE>0.02,valeurRevShare>142,valeurTCPS>24.5,valeurTDE<200,scoreValuationMeasures,scoreFinancialHighlights
3,TSM,0.066667,0.05,0.05,0.05,6.666667e-02,1.333333e-01,2.000000e-01,0.2,0.1,-2.000000e-01,0.1,2.000000e-01,0.1,0.1,0.05,0.066667,0.066667,4.000000e-01,1.000000
77,CAJ,0.066667,0.05,0.05,0.05,6.666667e-02,1.333333e-01,2.000000e-01,0.2,0.1,-2.000000e-01,0.1,2.000000e-01,0.1,0.1,0.05,0.066667,0.066667,4.000000e-01,1.000000
114,ASX,0.066667,0.05,0.05,0.05,-6.666667e-02,1.333333e-01,2.000000e-01,0.2,0.1,1.000000e-14,0.1,2.000000e-01,0.1,0.1,0.05,0.066667,0.066667,4.666667e-01,1.000000
44,WIT,0.066667,0.05,0.05,0.05,6.666667e-02,1.333333e-01,2.000000e-01,0.2,0.1,-2.000000e-01,0.1,2.000000e-01,0.1,0.1,-0.05,0.066667,0.066667,4.000000e-01,0.900000
74,UMC,0.066667,0.05,0.05,0.05,6.666667e-02,1.333333e-01,2.000000e-01,0.2,0.1,2.000000e-01,0.1,2.000000e-01,0.1,0.1,-0.05,0.066667,0.066667,8.000000e-01,0.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,DMRC,-0.066667,-0.05,-0.05,-0.05,1.000000e-14,1.000000e-14,1.000000e-14,-0.2,0.0,1.000000e-14,0.0,1.000000e-14,-0.1,-0.1,-0.05,-0.066667,0.066667,5.000000e-14,-0.666667
383,DM,-0.066667,-0.05,-0.05,-0.05,1.000000e-14,1.000000e-14,1.000000e-14,-0.2,0.0,1.000000e-14,0.0,1.000000e-14,-0.1,-0.1,-0.05,-0.066667,0.066667,5.000000e-14,-0.666667
292,AI,-0.066667,-0.05,-0.05,-0.05,1.000000e-14,1.000000e-14,1.000000e-14,-0.2,0.0,1.000000e-14,0.0,1.000000e-14,-0.1,-0.1,-0.05,-0.066667,0.066667,5.000000e-14,-0.666667
243,FOUR,-0.066667,-0.05,-0.05,-0.05,1.000000e-14,1.000000e-14,1.000000e-14,-0.2,0.1,1.000000e-14,0.0,1.000000e-14,-0.1,-0.1,-0.05,-0.066667,-0.066667,5.000000e-14,-0.700000


In [247]:
df2scores=df[['Symbol','scoreValuationMeasures','scoreFinancialHighlights' ]]
df2scores

,Symbol,scoreValuationMeasures,scoreFinancialHighlights
3,TSM,4.000000e-01,1.000000
77,CAJ,4.000000e-01,1.000000
114,ASX,4.666667e-01,1.000000
44,WIT,4.000000e-01,0.900000
74,UMC,8.000000e-01,0.900000
...,...,...,...
447,DMRC,5.000000e-14,-0.666667
383,DM,5.000000e-14,-0.666667
292,AI,5.000000e-14,-0.666667
243,FOUR,5.000000e-14,-0.700000


In [248]:
del df['scoreFinancialHighlights']
del df['scoreValuationMeasures']
df['score']=df.sum(axis=1)
df=df.sort_values(by=[ 'score', 'Symbol'], ascending=False)
df

,Symbol,1<valeurCR<3,EBITDA>21000376527,GProfit>18117636428,Revenue>97945222473,conditionTrailingPE,valeurEVEBITDA<10,valeurEVR<13,valeurLFCF>0,valeurOpM>-0.08,valeurPEG<1,valeurPM>-0.03,valeurPS<13,valeurROA>0.02,valeurROE>0.02,valeurRevShare>142,valeurTCPS>24.5,valeurTDE<200,score
74,UMC,0.066667,0.05,0.05,0.05,6.666667e-02,1.333333e-01,2.000000e-01,2.000000e-01,0.1,2.000000e-01,0.1,2.000000e-01,1.000000e-01,1.000000e-01,-0.05,0.066667,6.666667e-02,1.700000
114,ASX,0.066667,0.05,0.05,0.05,-6.666667e-02,1.333333e-01,2.000000e-01,2.000000e-01,0.1,1.000000e-14,0.1,2.000000e-01,1.000000e-01,1.000000e-01,0.05,0.066667,6.666667e-02,1.466667
150,SNX,0.066667,-0.05,-0.05,-0.05,1.000000e-14,1.333333e-01,2.000000e-01,2.000000e-01,0.1,2.000000e-01,0.1,2.000000e-01,1.000000e-01,1.000000e-01,0.05,0.066667,6.666667e-02,1.433333
3,TSM,0.066667,0.05,0.05,0.05,6.666667e-02,1.333333e-01,2.000000e-01,2.000000e-01,0.1,-2.000000e-01,0.1,2.000000e-01,1.000000e-01,1.000000e-01,0.05,0.066667,6.666667e-02,1.400000
77,CAJ,0.066667,0.05,0.05,0.05,6.666667e-02,1.333333e-01,2.000000e-01,2.000000e-01,0.1,-2.000000e-01,0.1,2.000000e-01,1.000000e-01,1.000000e-01,0.05,0.066667,6.666667e-02,1.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,SABRP,-0.066667,-0.05,-0.05,-0.05,1.000000e-14,1.000000e-14,1.000000e-14,-2.000000e-01,0.0,1.000000e-14,0.0,1.000000e-14,-1.000000e-01,-1.000000e-01,-0.05,-0.066667,-6.666667e-02,-0.800000
30,MRVL,0.066667,-0.05,-0.05,-0.05,-6.666667e-02,-1.333333e-01,-2.000000e-01,2.000000e-01,0.1,-2.000000e-01,0.0,-2.000000e-01,-1.000000e-01,-1.000000e-01,-0.05,-0.066667,6.666667e-02,-0.833333
119,ASAN,0.066667,-0.05,-0.05,-0.05,-6.666667e-02,1.333333e-01,-2.000000e-01,-2.000000e-01,0.0,1.000000e-14,0.0,-2.000000e-01,-1.000000e-01,-1.000000e-01,-0.05,-0.066667,6.666667e-02,-0.866667
179,DOCN,-0.066667,-0.05,-0.05,-0.05,1.000000e-14,-1.333333e-01,-2.000000e-01,1.000000e-14,0.1,-2.000000e-01,0.0,-2.000000e-01,1.000000e-14,1.000000e-14,-0.05,-0.066667,1.000000e-14,-0.966667


In [249]:
dfscore =df[['Symbol', 'score']]
dfscore

,Symbol,score
74,UMC,1.700000
114,ASX,1.466667
150,SNX,1.433333
3,TSM,1.400000
77,CAJ,1.400000
...,...,...
500,SABRP,-0.800000
30,MRVL,-0.833333
119,ASAN,-0.866667
179,DOCN,-0.966667


In [250]:
dfgraph=pd.merge(dfscore, df2scores,how="left",on="Symbol")
dfgraph

,Symbol,score,scoreValuationMeasures,scoreFinancialHighlights
0,UMC,1.700000,8.000000e-01,0.900000
1,ASX,1.466667,4.666667e-01,1.000000
2,SNX,1.433333,7.333333e-01,0.700000
3,TSM,1.400000,4.000000e-01,1.000000
4,CAJ,1.400000,4.000000e-01,1.000000
...,...,...,...,...
477,SABRP,-0.800000,5.000000e-14,-0.800000
478,MRVL,-0.833333,-8.000000e-01,-0.033333
479,ASAN,-0.866667,-3.333333e-01,-0.533333
480,DOCN,-0.966667,-7.333333e-01,-0.233333


In [251]:
dfgraph["scoreplot"]=dfgraph["score"]+3

# Création d'une fonction en vue d'obtenir un graphique intéractif et visualiser en haut à droite les meilleures entreprises suivant nos critères d'investissement

In [252]:

#on rajoute une diagonale pour voir les entreprises avec des scores équilibrés entre Valuation Measures et Financial Highlights
#on met des ronds de tailles différentes : les plus gros sont ceux pour les entreprises avec un score élevé
#on met les noms des entreprises sur chaque rond même si le graphe est intéractif
#L'intérêt d'avoir un graphe intéractif est de voir le score détaillé pour chaque entreprise et faire plus facilement nos choix d'investissement


def plot_scatter_graph(df,col_x,col_y, title_x = "axis_x",title_y="axis_y",title = "Graphe",
                       adapt_hover_to_axis_title = True, save_graph_name = "",annotation = None,
                       format_x = "{x}",format_y = "{y}", add_diagonal_line = False,col_color = None,
                       col_size = None,x_log=False) :
    
    # exemple format : {x:.0%}, {x:.2%}, {x:.2f}€
    
    if not "x" in format_x :
        format_x.replace("y", "x")
    if not "y" in format_y :
        format_x.replace("x", "y")
    
    df_graph = df.copy()
    
    if adapt_hover_to_axis_title :
        if title_x != "axis_x" :
            df_graph.rename({col_x: title_x}, axis=1, inplace=True)
            col_x = title_x
        if (title_y != "axis_y") and (isinstance(col_y, list) == False) :
            df_graph.rename({col_y: title_y}, axis=1, inplace=True)
            col_y = title_y

    if title_x == "axis_x" :
        title_x = col_x
    if (title_y == "axis_y") and (isinstance(col_y, list) == False):
        title_y = col_y
    
    fig = px.scatter(df_graph, x=col_x, y=col_y, text = annotation,color = col_color, size = col_size)
    
    fig.update_traces(textposition='top center')

    fig.update_layout({
            'plot_bgcolor': 'rgba(0,0,0,0)', # dernier nombre pour transparance
            'paper_bgcolor': 'rgba(0,0,0,0)',
        },
        hoverlabel=dict(
            #bgcolor="white", 
            font_size=12, 
            #font_family="Rockwell"
        ),
        xaxis={
            'title': title_x,
            },
        yaxis={'title':title_y},
        title={
            'text': title,
            #'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        hovermode = "x",
        font=dict(
            #family="Courier New, monospace",
            size=10,
            #color="#7f7f7f"
        )
    )
    
    if x_log :
        fig.update_layout(
            xaxis={
                'type':'log'
            }
        )

    if annotation is None :
        hovertemplate_str = ""
    else :
        hovertemplate_str = annotation + " : %{text} <br>"
    hovertemplate_str += col_y + " : %" + format_y + "<br>" + col_x + " : %" + format_x
    
    fig.update_traces(hovertemplate = hovertemplate_str
                     )
    
    if add_diagonal_line :
        fig.add_shape(type='line',
                    x0=0,
                    y0=0,
                    x1=df_graph[[col_x,col_y]].max().max(),
                    y1=df_graph[[col_x,col_y]].max().max(),
                    line=dict(color='Grey',width=1),
                    xref='x',
                    yref='y')

    # affiche pointillés entre point et les axes + affiche axes + ticks sur les axes
    fig.update_xaxes(showspikes=True, spikecolor = 'black', showline=True, linewidth=1,linecolor='black', ticks = "outside", tickwidth = 1, tickcolor = 'black',ticklen = 5)
    fig.update_yaxes(showspikes=True, spikecolor = 'black', showline=True, linewidth=1,linecolor='black', ticks = "outside", tickwidth = 1, tickcolor = 'black',ticklen = 5)
    
    if save_graph_name != "" :
        fig.write_html(save_graph_name + ".html")
    
    fig.show()

In [253]:
plot_scatter_graph(dfgraph,'scoreValuationMeasures','scoreFinancialHighlights', title_x = "score pour Valuation Measures",title_y="score pour Financial Highlights",title = "score des entreprises", annotation = "Symbol",add_diagonal_line = True,col_color = None,col_size = "scoreplot")
